In [2]:
import pandas as pd
import glob

import praw
import urllib.request
import json

from datetime import datetime
from datetime import date
import time


reddit = praw.Reddit(
    client_id="kRAJROlrIHaWMw",
    client_secret="f1EXDzFpTw1AryR_3urQ9O6I3AQ",
    user_agent="AITA_scrape",
)

In [3]:
def unix_to_date(unix):
    str_time = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(unix))
    date_time = datetime.strptime(str_time, "%Y-%m-%d %H:%M:%S")
    return date_time

def praw_recent_posts():
    posts = []
    for x in ["week", "month", "year"]:
        count = 0
        for submission in reddit.subreddit('amitheasshole').top(x, limit=1000):
            posts.append([
                submission.title,
                submission.id,
                unix_to_date(submission.created_utc),
                submission.link_flair_text,
                submission.selftext,
                submission.num_comments,
                submission.score,
                submission.upvote_ratio,
            ])
            count += 1
            if count % 300 == 0:
                print(x,": Completed",count)
    post_df = pd.DataFrame(posts, columns=["title", "id", "date", "verdict", "body", "num_comments", "score", "upvote_ratio"])
    post_df.drop_duplicates(subset="id", inplace=True)
    post_df.to_csv("data/raw/to_combine/praw_recent_%s.csv" % date.today().strftime("%Y%m%d"))
    return post_df

praw_rec_df = praw_recent_posts()
len(praw_rec_df)

week : Completed 300
week : Completed 600
week : Completed 900
month : Completed 300
month : Completed 600
month : Completed 900
year : Completed 300
year : Completed 600
year : Completed 900


2757

In [4]:
def combine_new_praw_data():
    files_to_combine = glob.glob("data/raw/to_combine/*.csv")
    temp_list = []

    for filename in files_to_combine:
        df = pd.read_csv(filename, index_col=0, header=0)
        temp_list.append(df)
    full_df = pd.concat(temp_list, axis=0, ignore_index=True)

    print(len(full_df))
    full_df.drop_duplicates(subset="id", inplace=True)
    full_df.reset_index(drop=True, inplace=True)
    print(len(full_df))

    full_df.to_csv("data/interim/aita_scrapped_full.csv")
    return full_df

full_df = combine_new_praw_data()
print(full_df.shape)
full_df

40111
36867
(36867, 8)


,verdict,body,num_comments,score,upvote_ratio,title,id,date
0,Not the A-hole,"I have a house I rent, and some of my neighbor...",324,4097,0.97,AITA for renting a house and telling my neighb...,g637y9,2020-04-22 15:31:14
1,Not the A-hole,tl;dr: i have large breasts that have caused m...,1089,19436,0.90,AITA my boyfriend wont let me get a breast red...,g5n4a3,2020-04-21 20:35:11
2,Not the A-hole,"I got in trouble at school this fall, I'm a ju...",2876,26316,0.93,AITA? My parents took most of my wardrobe away...,g5hta8,2020-04-21 15:56:36
3,NaN,[removed],200,0,0.48,AITA for being my trans sisters staunchest all...,g3yzy1,2020-04-19 01:18:47
4,Asshole,I'll try to make this short. I'm (38m) and my ...,805,3845,0.91,AITA for going on a week long vacation by myse...,g3y0xe,2020-04-19 00:11:52
...,...,...,...,...,...,...,...,...
36862,Not the A-hole,I'm using a throwaway because I don't want peo...,1303,14201,0.96,AITA for refusing to lend my friend a dress an...,i2a3iq,2020-08-02 10:43:03
36863,Not the A-hole,"A little over a month ago, my little brother w...",287,12683,0.99,AITA for wanting to bring the partner of my br...,hejlns,2020-06-23 18:04:51
36864,Not the A-hole,My (22F) sister is a single mom with a teenage...,1002,12498,0.98,AITA for letting my nephew stay with me withou...,i2dj11,2020-08-02 15:06:52
36865,Not the A-hole,Hello world of a Reddit! I am very conflicted ...,790,12124,0.97,AITA for refusing to let my daughter see her c...,i2dmx1,2020-08-02 15:13:32
